In [1]:
import pandas as pd
from emv.storage.storage import get_storage_client
from datetime import datetime
from IPython.display import Video

In [2]:
OUTDIR = "data"

In [3]:
metadata = '../data/metadata.hdf5'
with pd.HDFStore(metadata) as store:
    keys = store.keys()
    df = store.get(keys[0])  

In [4]:
def get_sequence_from_storage(seq_id: str, bucket_name='ioc') -> bytes:
    file_path = f"{OUTDIR}/{seq_id}.mp4"
    print(file_path)
    seq = df[df.seq_id == seq_id]
    print(seq.guid.values[0])

    ok = get_storage_client().download(bucket_name, 
        f"videos/{seq.guid.values[0]}/{seq_id}.mp4",
        f"{OUTDIR}/{seq_id}.mp4")
    print(seq.description.values[0])
    return Video(file_path)

In [7]:
seq_id = 'SEQ42710338'
get_sequence_from_storage(seq_id)

data/SEQ42710338.mp4
MDA42710309
Football ; Men's ; First Round ; Group A ; Men's Group A ; Foul ; TEAM (FRA)


In [ ]:
total_duration = 0

guid = df[df.seq_id == seq_id].guid.values[0]
df_norm = df[df.guid == guid].copy()

def seconds_to_minutes(seconds):
    minutes = seconds // 60
    seconds = seconds % 60 
    return int(minutes), int(seconds)

# Normalize the timestamps for the clips
df_norm.loc[:, 'start_ts'] = df_norm.start.apply(lambda x: (datetime.strptime(x, '%H:%M:%S:%f') - datetime(1900, 1, 1)).total_seconds())
df_norm.loc[:, 'end_ts'] = df_norm.end.apply(lambda x: (datetime.strptime(x, '%H:%M:%S:%f') - datetime(1900, 1, 1)).total_seconds())
df_norm.loc[:, 'end_ts'] = df_norm.end_ts - df_norm.start_ts.min()
df_norm.loc[:, 'start_ts'] = df_norm.start_ts - df_norm.start_ts.min()

df_norm = df_norm.sort_values(by='start_ts')

for i, row in df_norm.iterrows():
    print(row.seq_id, row.description, row.duration, total_duration, seconds_to_minutes(row.start_ts), '->', seconds_to_minutes(row.end_ts))
    total_duration += row.end_ts - row.start_ts